In [16]:
# !pip install bertopic

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bertopic import BERTopic
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import spacy
import seaborn as sns
from transformers import pipeline
import matplotlib.pyplot as plt
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter

In [8]:
df = pd.read_csv('8_9.5_all_necessary_only_add.moon.temp.des.csv')

In [9]:
df.head()

,aggressor,role,violence type,location,general location,severity,emotion,care,factor,description,response,month,day,day_of_week,is_weekend,Moon Phase,Full Moon,Precipitation,Temperature
0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,homelessness lack housing,verbal assault push shoving harassment head bu...,"Security Called, Law Enforcement Called, De-es...",1,9,1,0,0.915367,1,8.3,25.50
1,0.0,0.0,1.0,0.0,0.0,1.0,1.0,NaN,homelessness lack housing,verbal assault push shoving harassment head bu...,"Security Called, Law Enforcement Called, De-es...",1,9,1,0,0.915367,1,8.3,25.50
2,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,homelessness lack housing,verbal assault push shoving harassment head bu...,"Security Called, Law Enforcement Called, De-es...",1,9,1,0,0.915367,1,8.3,25.50
3,0.0,0.0,1.0,0.0,0.0,1.0,1.0,NaN,homelessness lack housing,verbal assault push shoving harassment head bu...,"Security Called, Law Enforcement Called, De-es...",1,9,1,0,0.915367,1,8.3,25.50
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,alter mental status inpatient bed unavailable,grab pinching scratch hair pull kicking hit be...,"Security Called, Law Enforcement Called, Physi...",2,6,1,0,0.865656,0,0.0,31.55


In [20]:
df2 = pd.read_csv('8_9.5_all_necessary_only_add.moon.temp.des.csv')
df2['severity'].value_counts()

,count
severity,
0.0,322
1.0,153
3.0,127
2.0,54


In [23]:
df3['severity'].value_counts()

,count
severity,
0.0,322
1.0,153
3.0,127
2.0,54


In [18]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Convert to lowercase and remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z\s]', '', str(text).lower())
    # Remove stopwords and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words]
    return ' '.join(tokens)

df["factor_clean"] = df["factor"].apply(clean_text)

all_words = ' '.join(df["factor_clean"]).split()
word_freq = Counter(all_words)

# Remove rows with empty or garbage "factor" entries
df["factor"] = df["factor"].fillna("")  # Replace NaN with empty strings
df = df[~df["factor"].str.contains(r"^[,\s]+$", regex=True)]  # Remove rows with only commas/spaces
df = df[df["factor"].str.strip().str.len() > 3]  # Keep rows with meaningful text (>3 characters)
# Clean text (expand your preprocessing)
def clean_text(text):
    text = re.sub(r"[^a-zA-Z\s]", "", str(text).lower())  # Remove punctuation
    tokens = [word for word in text.split() if word not in stop_words]
    return " ".join(tokens)

df["factor_clean"] = df["factor"].apply(clean_text)

# Train BERTopic on cleaned data
topic_model = BERTopic(language="english", min_topic_size=10)
topics, _ = topic_model.fit_transform(df["factor_clean"])
df["topic"] = topics

# Get topic keywords
topic_info = topic_model.get_topic_info()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [29]:
df3 = pd.read_csv('8_9.5_all_necessary_only_add.moon.temp.des.csv')

In [30]:
# Clean and process 'factor' column ============================================
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', str(text).lower())
    tokens = [lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words]
    return ' '.join(tokens)

# Clean factor column
df3["factor_clean"] = df3["factor"].apply(clean_text)

# Filter with severity preservation
initial_severity = df3["severity"].value_counts(dropna=False)
df3 = df3[df3["factor_clean"].str.strip().str.len() > 1]
df3 = df3[df3["factor_clean"].str.strip().ne("")]

# Train BERTopic on factor and get labels
factor_topic_model = BERTopic(language="english", min_topic_size=10)
topics, _ = factor_topic_model.fit_transform(df3["factor_clean"])
df3["topic"] = topics

# Create topic labels from model's keyword representations
topic_info = factor_topic_model.get_topic_info()
topic_label_map = dict(zip(topic_info.Topic, topic_info.Name))
df3["topic_label"] = df3["topic"].map(topic_label_map)

# Clean and process 'description' column =======================================
# Reuse same cleaning function
df3["description_clean"] = df3["description"].apply(clean_text)

# Filter with severity check
print("\nBefore description cleaning:")
print(df3["severity"].value_counts(dropna=False))

df3 = df3[df3["description_clean"].str.strip().ne("")]

print("\nAfter description cleaning:")
print(df3["severity"].value_counts(dropna=False))

# Train separate BERTopic model for descriptions
desc_topic_model = BERTopic(language="english", min_topic_size=10)
desc_topics, _ = desc_topic_model.fit_transform(df3["description_clean"])
df3["desc_topic"] = desc_topics

# Create description topic labels
desc_topic_info = desc_topic_model.get_topic_info()
desc_topic_label_map = dict(zip(desc_topic_info.Topic, desc_topic_info.Name))
df3["desc_topic_label"] = df3["desc_topic"].map(desc_topic_label_map)

# Final checks =================================================================
print("\nFinal severity distribution:")
print(df3["severity"].value_counts(dropna=False))

print("\nSample topic labels:")
print(df3[["topic", "topic_label", "desc_topic", "desc_topic_label"]].sample(3))


Before description cleaning:
severity
NaN    975
0.0    322
1.0    153
3.0    127
2.0     54
Name: count, dtype: int64

After description cleaning:
severity
NaN    975
0.0    322
1.0    153
3.0    127
2.0     54
Name: count, dtype: int64

Final severity distribution:
severity
NaN    975
0.0    322
1.0    153
3.0    127
2.0     54
Name: count, dtype: int64

Sample topic labels:
      topic topic_label  desc_topic                    desc_topic_label
1595      0    0_nan___           5  5_intimidation_tone_record_consent
1599      0    0_nan___           3        3_threat_violence_fist_fluid
899       0    0_nan___          -1             -1_pt_staff_patient_bed


In [31]:
df3.head()

,aggressor,role,violence type,location,general location,severity,emotion,care,factor,description,...,Moon Phase,Full Moon,Precipitation,Temperature,factor_clean,topic,topic_label,description_clean,desc_topic,desc_topic_label
0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,homelessness lack housing,verbal assault push shoving harassment head bu...,...,0.915367,1,8.3,25.50,homelessness lack housing,7,7_housing_homelessness_lack_,verbal assault push shoving harassment head bu...,21,21_harassment_verbal_assault_posture
1,0.0,0.0,1.0,0.0,0.0,1.0,1.0,NaN,homelessness lack housing,verbal assault push shoving harassment head bu...,...,0.915367,1,8.3,25.50,homelessness lack housing,7,7_housing_homelessness_lack_,verbal assault push shoving harassment head bu...,21,21_harassment_verbal_assault_posture
2,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,homelessness lack housing,verbal assault push shoving harassment head bu...,...,0.915367,1,8.3,25.50,homelessness lack housing,7,7_housing_homelessness_lack_,verbal assault push shoving harassment head bu...,21,21_harassment_verbal_assault_posture
3,0.0,0.0,1.0,0.0,0.0,1.0,1.0,NaN,homelessness lack housing,verbal assault push shoving harassment head bu...,...,0.915367,1,8.3,25.50,homelessness lack housing,7,7_housing_homelessness_lack_,verbal assault push shoving harassment head bu...,21,21_harassment_verbal_assault_posture
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,alter mental status inpatient bed unavailable,grab pinching scratch hair pull kicking hit be...,...,0.865656,0,0.0,31.55,alter mental status inpatient bed unavailable,12,12_psychiatric_bed_care_removal,grab pinching scratch hair pull kicking hit be...,23,23_verbal_pinching_hair_assault


In [32]:
df3.to_csv('tmp.csv', index=False)